In [1]:
suppressMessages(library(SPEI))

In [102]:
sapply(c("mswep", "chirps"), function(ds) {
    df = read.csv(paste0("data/", ds, "_monthly.csv"))
    df_ts <- ts(df$pr, start = sapply(strsplit(df$time[1], "-")[[1]], as.integer)[1:2], frequency = 12)
    res_df <- sapply(c(1,3,6), function(nm) {
        spi_m <- spi(df_ts, scale = nm, ref.start = c(1990,1), ref.end = c(2020,12))
        spi_m$fitted
    })
    write.csv(setNames(data.frame(df$time, res_df), c("time", "spi_1", "spi_3", "spi_6")), file = paste0("data/",ds,"_spi-monthly.csv"), row.names = F)
})

$mswep
NULL

$chirps
NULL